<a href="https://colab.research.google.com/github/JKH-ML/python/blob/main/7_K_%EC%B5%9C%EA%B7%BC%EC%A0%91_%EC%9D%B4%EC%9B%83_(K_NN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-최근접 이웃 (K-Nearest Neighbors, K-NN)

## 개념

K-NN은 **지도 학습 (Supervised Learning)** 방식의 가장 직관적인 분류 알고리즘입니다.  
입력 데이터가 주어졌을 때, **가장 가까운 K개의 훈련 샘플을 기준으로 다수결** 혹은 평균 등을 사용해 예측합니다.

- 거리 기반 분류 알고리즘
- 별도의 학습 과정이 필요 없음 (Lazy learning)
- 분류와 회귀 모두 사용 가능

---

## 수식

### 유클리드 거리 (Euclidean Distance)

두 점 $x = (x_1, x_2, ..., x_n)$, $y = (y_1, y_2, ..., y_n)$ 사이의 거리는 다음과 같이 계산됩니다:

$$
d(x, y) = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}
$$

---

## 알고리즘 절차

1. 새로운 샘플 $x$가 주어짐
2. 학습 데이터 전체와의 거리를 계산
3. 거리 기준으로 가장 가까운 K개의 이웃 선택
4. 이웃의 라벨 중 **가장 많은 클래스**를 예측값으로 사용 (분류인 경우)

---

## 장점

- 구현이 간단하고 직관적
- 새로운 데이터에 유연하게 대응 가능

## 단점

- 고차원에서는 거리 개념이 희석되어 성능 저하
- 계산량이 많고 느림 (모든 데이터와 거리 계산)


In [1]:
import numpy as np
import plotly.graph_objects as go
from collections import Counter

# 샘플 데이터 생성 (이진 분류용)
np.random.seed(42)
num_samples = 100
X_class0 = np.random.randn(num_samples, 2) + np.array([-2, 0])
X_class1 = np.random.randn(num_samples, 2) + np.array([2, 0])
X_train = np.vstack((X_class0, X_class1))
y_train = np.array([0]*num_samples + [1]*num_samples)

# K-NN 함수 정의
def knn_predict(X_train, y_train, x_query, k=5):
    distances = np.sqrt(np.sum((X_train - x_query)**2, axis=1))
    nearest_indices = distances.argsort()[:k]
    nearest_labels = y_train[nearest_indices]
    most_common = Counter(nearest_labels).most_common(1)
    return most_common[0][0]

# 테스트 예시
x_query = np.array([0, 1])
y_pred = knn_predict(X_train, y_train, x_query, k=5)

# 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_class0[:, 0], y=X_class0[:, 1],
                         mode='markers', name='Class 0', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=X_class1[:, 0], y=X_class1[:, 1],
                         mode='markers', name='Class 1', marker=dict(color='red')))
fig.add_trace(go.Scatter(x=[x_query[0]], y=[x_query[1]],
                         mode='markers+text', name='Query', text=[f'Pred: {y_pred}'],
                         marker=dict(color='black', size=12), textposition='top center'))
fig.update_layout(title='K-NN Classification (k=5)', xaxis_title='X1', yaxis_title='X2')
fig.show()


# K-NN 코드 설명

## 1. 데이터 생성

```python
X_class0 = np.random.randn(num_samples, 2) + np.array([-2, 0])
X_class1 = np.random.randn(num_samples, 2) + np.array([2, 0])
```

- 클래스 0과 클래스 1의 데이터를 2차원 평면 상에 각각 생성합니다.
- 각각 평균이 다르게 설정되어 있어 분리 가능하게 됩니다.

---

## 2. 거리 계산 및 이웃 선택

```python
distances = np.sqrt(np.sum((X_train - x_query)**2, axis=1))
nearest_indices = distances.argsort()[:k]
```

- 모든 학습 샘플과 테스트 점 `x_query` 사이의 유클리드 거리를 계산합니다.
- 거리 기준으로 오름차순 정렬 후, 가장 가까운 `k`개의 인덱스를 가져옵니다.

---

## 3. 다수결 투표 (분류)

```python
nearest_labels = y_train[nearest_indices]
most_common = Counter(nearest_labels).most_common(1)
```

- 가까운 샘플들의 라벨 중 가장 많은 값을 예측값으로 사용합니다.
- `Counter`를 사용하여 효율적으로 다수 클래스를 찾습니다.

---

## 4. 시각화

- 파란색: 클래스 0
- 빨간색: 클래스 1
- 검정색 점: 테스트할 새 샘플
- 텍스트: 예측된 클래스 출력

Plotly를 사용하여 2차원 평면에 시각화했습니다.
